In [1]:
import logging
import warnings

import numpy as np
import pandas as pd
import copy
import shapely
import xarray as xr
from shapely.geometry import LineString
from shapely.ops import cascaded_union
from typing import Dict, List, Tuple

import FINE.spagat.utils as spu
import FINE.spagat.dataset as spd
import FINE.spagat.representation as spr

logger_representation = logging.getLogger('spagat_representation')
from shapely.geometry import Polygon, MultiPolygon

In [2]:
#DICT
sub_to_sup_region_id_dict = {'01_reg_02_reg': ['01_reg','02_reg'], 
                             '03_reg_04_reg': ['03_reg','04_reg']}

#SDS
component_list = ['source_comp','sink_comp', 'transmission_comp']  
space_list = ['01_reg','02_reg','03_reg','04_reg']
TimeStep_list = ['T0','T1']
Period_list = [0]

## ts variable data
operationRateMax = np.array([ [ [ [3, 3, 3, 3] for i in range(2)] ],

                      [ [[np.nan, np.nan, np.nan, np.nan] for i in range(2)] ], 

                      [ [[np.nan, np.nan, np.nan, np.nan] for i in range(2)] ]

                      ])

operationRateMax_da = xr.DataArray(operationRateMax, 
                              coords=[component_list, Period_list, TimeStep_list, space_list], 
                              dims=['component', 'Period', 'TimeStep','space'])

operationRateFix = np.array([ [ [[np.nan, np.nan, np.nan, np.nan] for i in range(2)] ], 

                      [ [ [5, 5, 5, 5] for i in range(2)] ],

                      [ [[np.nan, np.nan, np.nan, np.nan] for i in range(2)] ]

                      ])

operationRateFix_da = xr.DataArray(operationRateFix, 
                              coords=[component_list, Period_list, TimeStep_list, space_list], 
                              dims=['component', 'Period', 'TimeStep','space'])

## 1d variable data
capacityMax_1d = np.array([ [15,  15,  15, 15],
                        [np.nan] *4, 
                        [np.nan] *4, 
                      ])

capacityMax_1d_da = xr.DataArray(capacityMax_1d, 
                          coords=[component_list, space_list], 
                          dims=['component', 'space'])

capacityFix_1d = np.array([ [np.nan] *4, 
                       [5,  5,  5, 5],
                        [np.nan] *4, 
                      ])

capacityFix_1d_da = xr.DataArray(capacityFix_1d, 
                          coords=[component_list, space_list], 
                          dims=['component', 'space'])

## 2d variable data
capacityMax_2d = np.array([ [[np.nan] * 4 for i in range(4)], 

                    [[np.nan] * 4 for i in range(4)],

                     [[ 0,  5,  5, 5],
                      [ 5,  0,  5, 5],
                      [ 5,  5,  0, 5],
                      [ 5,  5,  5, 0]]
                    ])

capacityMax_2d_da = xr.DataArray(capacityMax_2d, 
                          coords=[component_list, space_list, space_list], 
                          dims=['component', 'space', 'space_2'])

locationalEligibility_2d = np.array([ [[np.nan] * 4 for i in range(4)], 

                    [[np.nan] * 4 for i in range(4)],

                     [[ 0,  1,  1, 1],
                      [ 0,  0,  1, 1],
                      [ 0,  0,  0, 1],
                      [ 0,  0,  0, 0]]
                    ])

locationalEligibility_2d_da = xr.DataArray(locationalEligibility_2d, 
                          coords=[component_list, space_list, space_list], 
                          dims=['component', 'space', 'space_2'])

test_ds = xr.Dataset({'ts_operationRateMax': operationRateMax_da, 
                    'ts_operationRateFix': operationRateFix_da,
                    '1d_capacityMax': capacityMax_1d_da, 
                    '1d_capacityFix': capacityFix_1d_da, 
                    '2d_capacityMax': capacityMax_2d_da, 
                    '2d_locationalEligibility': locationalEligibility_2d_da
                    })    

sds = spd.SpagatDataset()
sds.xr_dataset = test_ds            

test_geometries = [Polygon([(0,0), (2,0), (2,2), (0,2)]),
                Polygon([(2,0), (4,0), (4,2), (2,2)]),
                Polygon([(0,0), (4,0), (4,4), (0,4)]),
                Polygon([(0,0), (1,0), (1,1), (0,1)])]   

sds.add_objects(description ='gpd_geometries',   #NOTE: not sure if it is ok to call another function here
            dimension_list =['space'], 
            object_list = test_geometries)

In [13]:
aggregation_function_dict = {'operationRateMax': ('mean', None), 
                'operationRateFix': ('sum', None), 
                'capacityMax': ('sum', None), 
                'capacityFix': ('sum', None), 
                'locationalEligibility': ('bool', None)}

In [14]:
aggregated_sds = spr.aggregate_based_on_sub_to_sup_region_id_dict(sds,
                                            sub_to_sup_region_id_dict,
                                            aggregation_function_dict=aggregation_function_dict)

In [15]:
aggregated_sds.xr_dataset

<xarray.Dataset>
Dimensions:                   (Period: 1, TimeStep: 2, component: 3, space: 2, space_2: 2)
Coordinates:
  * component                 (component) <U17 'source_comp' ... 'transmissio...
  * Period                    (Period) int32 0
  * TimeStep                  (TimeStep) <U2 'T0' 'T1'
  * space                     (space) <U13 '01_reg_02_reg' '03_reg_04_reg'
  * space_2                   (space_2) <U13 '01_reg_02_reg' '03_reg_04_reg'
Data variables:
    ts_operationRateMax       (component, Period, TimeStep, space) float64 3....
    ts_operationRateFix       (component, Period, TimeStep, space) float64 0....
    1d_capacityMax            (component, space) float64 30.0 30.0 ... 0.0 0.0
    1d_capacityFix            (component, space) float64 0.0 0.0 ... 0.0 0.0
    2d_capacityMax            (component, space, space_2) float64 0.0 ... 0.0
    2d_locationalEligibility  (component, space, space_2) float64 0.0 ... 0.0
    gpd_geometries            (space) object POLYGON ((2 0, 0 0, 0 2, 2 2, 4 ...
    gpd_centroids             (space) object POINT (2 1) POINT (2 2)